In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!git clone https://github.com/LEVIII007/Sign_language_final.git

Cloning into 'Sign_language_final'...
remote: Enumerating objects: 8886, done.
remote: Counting objects: 100% (8886/8886), done.
remote: Compressing objects: 100% (8884/8884), done.
remote: Total 8886 (delta 0), reused 8883 (delta 0), pack-reused 0
Receiving objects: 100% (8886/8886), 17.15 MiB | 39.38 MiB/s, done.


In [3]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import os

2024-02-10 21:56:30.580514: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-10 21:56:30.580641: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-10 21:56:30.734267: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
)

val_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 128

train_generator = train_datagen.flow_from_directory(
    '/kaggle/working/Sign_language_final/',
    target_size=(80,80),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale',
    classes= ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'Y', 'blank', 'hello', 'byee','how_are_you','you','love' ],    subset='training',  # Use 'training' to fetch only specified classes
    seed=42  # Optional seed for reproducibility
)

validation_generator = val_datagen.flow_from_directory(
    '/kaggle/working/Sign_language_final/',
    target_size=(80,80),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale',
    classes= ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'Y', 'blank', 'hello', 'byee','how_are_you','you','love' ],
    subset='training',  # Use 'validation' to fetch only specified classes
    seed=42  # Optional seed for reproducibility
)


Found 6653 images belonging to 26 classes.
Found 6653 images belonging to 26 classes.


In [5]:
class_names = list(train_generator.class_indices.keys())
print(class_names)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'Y', 'blank', 'hello', 'byee', 'how_are_you', 'you', 'love']


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 475.2/475.2 MB 91.7 MB/s eta 0:00:01^C:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 475.2/475.2 MB 91.7 MB/s eta 0:00:01Note: you may need to restart the kernel to use updated packages.


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential()

# Convolutional layers
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', input_shape=(80, 80, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(1024, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(1024, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())

# Fully connected layers
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(26, activation='softmax'))  # Output layer for 31 classes

# You can add more hidden layers as needed

# Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 78, 78, 256)       2560      
                                                                 
 max_pooling2d (MaxPooling2  (None, 39, 39, 256)       0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 39, 39, 256)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 37, 37, 512)       1180160   
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 18, 18, 512)       0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 18, 18, 512)       0

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 78, 78, 256)       2560      
                                                                 
 max_pooling2d (MaxPooling2  (None, 39, 39, 256)       0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 39, 39, 256)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 37, 37, 512)       1180160   
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 18, 18, 512)       0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 18, 18, 512)       0

In [8]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

In [9]:
!rm -rf Logs
logdir = os.path.join("Logs")
tensorboard_callback = TensorBoard(log_dir=logdir)

In [10]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=100,
    callbacks=[tensorboard_callback]
)

Epoch 1/100


2024-02-10 21:57:28.162978: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1707602258.994706     125 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


51/51 [==============================] - 72s 817ms/step - loss: 3.2216 - accuracy: 0.0527
Epoch 2/100
51/51 [==============================] - 18s 354ms/step - loss: 3.0828 - accuracy: 0.0754
Epoch 3/100
51/51 [==============================] - 18s 359ms/step - loss: 2.7055 - accuracy: 0.1910
Epoch 4/100
51/51 [==============================] - 19s 372ms/step - loss: 2.1051 - accuracy: 0.3274
Epoch 5/100
51/51 [==============================] - 19s 379ms/step - loss: 1.1868 - accuracy: 0.6077
Epoch 6/100
51/51 [==============================] - 19s 371ms/step - loss: 0.4899 - accuracy: 0.8337
Epoch 7/100
51/51 [==============================] - 19s 369ms/step - loss: 0.2397 - accuracy: 0.9195
Epoch 8/100
51/51 [==============================] - 19s 370ms/step - loss: 0.1489 - accuracy: 0.9574
Epoch 9/100
51/51 [==============================] - 19s 374ms/step - loss: 0.0872 - accuracy: 0.9746
Epoch 10/100
51/51 [==============================] - 19s 373ms/step - loss: 0.0671 - accuracy

In [12]:
model_json = model.to_json()
with open("/kaggle/working/sl.json",'w') as json_file:
    json_file.write(model_json)
# model.save("/kaggle/working/model_final.h5")
model.save("/kaggle/working/model_final.keras")